# **Trabalho Avaliativo 2 - Inteligência Artificial**
**Acadêmicos:**
- Guilherme Farias Stefani
- Henrique Baptista de Oliveira
- Henrique Cardoso Zanette
- Jhonata Saraiva Peres
- Lucas Dellatorre de Freitas


___

### **Bibliotecas Python nescessarias para utilização deste notebook:**

- **Pandas:** Para leitura do DataSet

- **Unidecode:** Para normalização dos textos

- **Regex:** Para normalização dos textos
     
- **Nltk:** Para executar os algoritmos de machine learning

- **Ploty:** Para gráficos e afins


*Rodar no powershell como administrador*

> - pip install pandas
> - pip install unidecode
> - pip install regex
> - pip install nltk
> - pip install plotly==5.11.0

*Import das bibliotecas*

In [91]:
import pandas as pd
import regex as re
import unidecode
import nltk
import plotly.express as px

___

### **Repositório de arquivos, datasets e dados de normalização**

*Localização dos arquivos de resources*

In [92]:
filenameDataSet = 'resources/DataSet.xlsx'
filenameRegexToBeApplied =  'resources/regexToBeApplied'
filenameStopWords = 'resources/stopWords'
fileNameDeParas = 'resources/dePara'

___

### **Dataset e amostras**

*Amostra completa do dataset*

In [93]:
dataSet = pd.ExcelFile(filenameDataSet)

In [94]:
dataSetRaw = pd.read_excel(dataSet, sheet_name='DebateGovRS')

*Amostra do dataset apenas com retweets*

In [95]:
retweets = pd.read_excel(dataSet, sheet_name='RT')

*Amostra do dataset classificado/rotulado*

In [96]:
rotulados = pd.read_excel(dataSet, sheet_name='ROTULADO')

fig = px.bar(rotulados['text'], x=rotulados['rotulo'])
fig.show()

In [97]:
treino = pd.read_excel(dataSet, sheet_name='TREINO_70%')

fig = px.bar(treino['text'], x=treino['rotulo'])
fig.show()

___

### **Classificação e Limpeza dos Dados**

- Inicialmente, retiramos todos os retweets presentes no dataset;

- Em seguida, definimos os tipos de classificação dos dados, conforme abaixo:
>   - **Contexto:** Precisa de contexto para ser entendido
>   - **Positivo:**  Mensagens positivas: Elogios, enaltecimento de candidato, etc.
>   - **Negativo:** Mensagens negativas: Ofensas, xingamentos, desmerecer propostas.
>   - **Ironia:** Sentido contrario da mensagem
>   - **Neutro:** Nem negativo nem positivo. Somente descritivo (narração de fatos)
>   - **Confuso:** Não foi possível interpretar o significado do texto
>   - **Dividir:** Mensagens positivas e negativas com possibilidade de quebra para geração de novos textos

- Após a definição dos tipos de classificação, começamos a classificar individualmente e manualmente, um por um;

- Depois da classificação individual, realizamos uma revisão geral em grupo, onde foram ponderadas e debatidas as classificações, também de forma manual, percorrendo por todas as linhas do dataset;

___

### **Normalização dos dados**

Para normalizar os dados, utilizamos de algumas ferramentas e técnicas a fim de garantir um melhor resultado pelo algoritmo, assim realizamos os seguintes passos:


*Inicialmente, todo o input foi transformado para caixa baixa, as quebras de linha foram removidas do texto e os espaços duplos foram transformados em espaço único*

In [98]:
def toLower(input):
    return input.lower()

def removeLinesBreaks(input):
    return re.sub('\\\\n{1,}|\\\\r{1,}', ' ', input, re.IGNORECASE, re.MULTILINE)

def removeDoubleSpaces(input):
    return re.sub('\\s{2,}', ' ', input, re.IGNORECASE, re.MULTILINE)

def aplyRegex(input):
    regexs = open(filenameRegexToBeApplied, 'r', encoding='UTF-8', newline='\n')
    for regex in regexs.readlines():  
        teste = re.sub(regex, ' ', input, re.IGNORECASE, re.MULTILINE)
        input = teste
    return input

*Em seguida, utilizando um arquivo customizado desenvolvido pelo grupo, transformamos palavras e nomes com diversas variações para uma forma única, como por exemplo, Você ou vc para voce*

In [99]:
def dePara(input):
    deParas = open(fileNameDeParas, 'r', encoding='UTF-8', newline='\n')
    for dePara in deParas.readlines():
        dePara = dePara.split('(-*-)')
        input = re.sub(dePara[0], dePara[1], input, re.IGNORECASE, re.MULTILINE)
    return input

*Após a transformação das palavras, utilizamos a técnica de stop words para remover as palavras que não trazem significado para o texto, como por exemplo, os artigos: a, as, o, os*

In [100]:
def removeStopWords(input):
    stopWrods = open(filenameStopWords, 'r', encoding='UTF-8', newline='\r\n')
    for stopWord in stopWrods.readlines():
        stopWord = unidecode.unidecode(stopWord)
        stopWord = re.sub('\r\n', '', stopWord)
        input = re.sub(stopWord, '', input, re.IGNORECASE, re.MULTILINE)
    return input

*Ainda no processo de normalização, removemos todos os acentos das palavras do dataset, a fim de padronizar os dados para posterior análise pelo algoritmo*

In [101]:
def removeAcentuacao(input):
    return unidecode.unidecode(input)

In [102]:
listAux = []

for line in rotulados.values:
    line[1] = removeDoubleSpaces(dePara(removeLinesBreaks(removeAcentuacao(removeStopWords(toLower(unidecode.unidecode(line[1])))))))
    listAux.append(line)
  
rotulados = pd.DataFrame(listAux)


___

### **Criação dos conjuntos de treino, teste e validação**

- A partir de todo o conjunto de dados classificado, foram mantidos apenas os valores positivos e negativos (Ironias e confusões foram removidas e divisões possíveis não foram realizadas)

- Ordenamos por ordem alfabetica para retirar os textos duplicados do conjunto de dados

- Utilizando o tipo de dado com menor amostras (POSITIVO), foi determinado 70% de suas linhas, a quantidade necessária para o conjunto de treino.

- Antes de obter os dados e alocar-los no conjunto de treino, sua ordenação foi aleatorizada para abrangir uma quantidade maior de textos.

- Após aleatorizar foram colocados no conjunto de treino, com quantidades iguais de positivos e negativos.

- Posteriormente foram criados os conjuntos de testes e validação, ambos com 15% cada, do total de amostras do tipo POSITIVO (com menor amostras), com os valores também ordenados de forma aleatória, mas balanceados entre positivos e negativos

___

### **Bag of Words**

In [103]:
from sklearn.feature_extraction.text import CountVectorizer
#todosPositivos = pd.read_excel(dataSet, sheet_name='TODOS_POSITIVOS')
tweets = treino["text"].values
classificacao = treino["rotulo"].values
vectorizer = CountVectorizer(analyzer = "word")
freq_tweets = vectorizer.fit_transform(tweets)

testes = pd.read_excel(dataSet, sheet_name='TESTE_15%')
testes = testes["text"].values

Biblioteca de métricas e cross-validation

In [104]:
from sklearn import metrics
from sklearn.model_selection import cross_val_predict
import re

NAIVE BAYES

In [105]:
from sklearn.naive_bayes import MultinomialNB
modelo = MultinomialNB()
modelo.fit(freq_tweets, classificacao)

freq_testes = vectorizer.transform(testes)
modelo.predict(freq_testes)
resultados = cross_val_predict(modelo, freq_tweets, classificacao, cv = 20)
metrics.accuracy_score(classificacao, resultados)
print(metrics.classification_report(classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))


              precision    recall  f1-score   support

    POSITIVO       0.85      0.79      0.82       248
    NEGATIVO       0.80      0.86      0.83       248

    accuracy                           0.83       496
   macro avg       0.83      0.83      0.83       496
weighted avg       0.83      0.83      0.83       496



LOGISTIC REGRESSION

In [106]:
from sklearn.linear_model import LogisticRegression
modeloLR = LogisticRegression(C = 2, max_iter = 1000, n_jobs=-1)
modeloLR.fit(freq_tweets, classificacao)
resultados = cross_val_predict(modeloLR, freq_tweets, classificacao, cv = 10)
metrics.accuracy_score(classificacao, resultados)
print(metrics.classification_report(classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))

              precision    recall  f1-score   support

    POSITIVO       0.82      0.83      0.82       248
    NEGATIVO       0.83      0.81      0.82       248

    accuracy                           0.82       496
   macro avg       0.82      0.82      0.82       496
weighted avg       0.82      0.82      0.82       496



KNN

In [110]:
from sklearn.neighbors import KNeighborsClassifier

for k in [7,11,13]:

    modeloKnn = KNeighborsClassifier(n_neighbors=k,algorithm='brute') #Using brute-force algorithm for quicker computation.
    modeloKnn.fit(freq_tweets, classificacao)
    resultados = cross_val_predict(modeloKnn, freq_tweets, classificacao, cv = 10)
    metrics.accuracy_score(classificacao, resultados)
    print(metrics.classification_report(classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))

              precision    recall  f1-score   support

    POSITIVO       0.82      0.22      0.35       248
    NEGATIVO       0.55      0.95      0.70       248

    accuracy                           0.59       496
   macro avg       0.69      0.59      0.52       496
weighted avg       0.69      0.59      0.52       496

              precision    recall  f1-score   support

    POSITIVO       0.79      0.18      0.30       248
    NEGATIVO       0.54      0.95      0.69       248

    accuracy                           0.57       496
   macro avg       0.66      0.57      0.49       496
weighted avg       0.66      0.57      0.49       496

              precision    recall  f1-score   support

    POSITIVO       0.77      0.17      0.27       248
    NEGATIVO       0.53      0.95      0.68       248

    accuracy                           0.56       496
   macro avg       0.65      0.56      0.48       496
weighted avg       0.65      0.56      0.48       496



MLP

In [108]:
from sklearn.neural_network import MLPClassifier

modeloMLP = MLPClassifier()
modeloMLP.fit(freq_tweets, classificacao)
resultados = cross_val_predict(modeloMLP, freq_tweets, classificacao, cv = 10)
metrics.accuracy_score(classificacao, resultados)
print(metrics.classification_report(classificacao, resultados, target_names=['POSITIVO', 'NEGATIVO']))


              precision    recall  f1-score   support

    POSITIVO       0.85      0.79      0.82       248
    NEGATIVO       0.81      0.86      0.83       248

    accuracy                           0.83       496
   macro avg       0.83      0.83      0.83       496
weighted avg       0.83      0.83      0.83       496

